In [40]:
!pip install pyspark


In [41]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
spark = SparkSession.builder.appName("MySparkApp").getOrCreate()

In [42]:
df=spark.read.format("csv").option("header", "true").load("large_customer_transactions.csv")
df.show()

+--------------+-----------+------+-----------+----------------+--------------+
|transaction_id|customer_id|amount|   category|transaction_date|payment_method|
+--------------+-----------+------+-----------+----------------+--------------+
|          1002|       C002|   826|  Groceries|      2024-07-23|        PayPal|
|          1004|       C004|   346|     Travel|      2024-09-25| Bank Transfer|
|          1006|       C006|  1006|  Furniture|      2024-11-27|    Debit Card|
|          1008|       C008|   730|Electronics|      2024-01-01|          Cash|
|          1010|       C010|   176|   Clothing|      2024-03-03|   Credit Card|
|          1012|       C012|   117|     Health|      2024-05-05|        PayPal|
|          1014|       C014|   628|      Books|      2024-07-07| Bank Transfer|
|          1016|       C016|   594|  Groceries|      2024-09-09|    Debit Card|
|          1018|       C018|   211|     Travel|      2024-11-11|          Cash|
|          1020|       C020|   218|  Fur

In [43]:
df1=spark.read.format("csv").option("header", "true").load("/content/large_customer_transactions (1).csv")
df1.show()

+--------------+-----------+------+-----------+----------------+--------------+
|transaction_id|customer_id|amount|   category|transaction_date|payment_method|
+--------------+-----------+------+-----------+----------------+--------------+
|          1001|       C001|   168|Electronics|      2024-06-22|    Debit Card|
|          1003|       C003|   690|   Clothing|      2024-08-24|          Cash|
|          1005|       C005|   622|     Health|      2024-10-26|   Credit Card|
|          1007|       C007|   540|      Books|      2024-12-28|        PayPal|
|          1009|       C009|   900|  Groceries|      2024-02-02| Bank Transfer|
|          1011|       C011|   939|     Travel|      2024-04-04|    Debit Card|
|          1013|       C013|   552|  Furniture|      2024-06-06|          Cash|
|          1015|       C015|   966|Electronics|      2024-08-08|   Credit Card|
|          1017|       C017|   935|   Clothing|      2024-10-10|        PayPal|
|          1019|       C019|   137|     

In [44]:
df_new=df.unionByName(df1)
df_new.show()
print(df_new.count())

+--------------+-----------+------+-----------+----------------+--------------+
|transaction_id|customer_id|amount|   category|transaction_date|payment_method|
+--------------+-----------+------+-----------+----------------+--------------+
|          1002|       C002|   826|  Groceries|      2024-07-23|        PayPal|
|          1004|       C004|   346|     Travel|      2024-09-25| Bank Transfer|
|          1006|       C006|  1006|  Furniture|      2024-11-27|    Debit Card|
|          1008|       C008|   730|Electronics|      2024-01-01|          Cash|
|          1010|       C010|   176|   Clothing|      2024-03-03|   Credit Card|
|          1012|       C012|   117|     Health|      2024-05-05|        PayPal|
|          1014|       C014|   628|      Books|      2024-07-07| Bank Transfer|
|          1016|       C016|   594|  Groceries|      2024-09-09|    Debit Card|
|          1018|       C018|   211|     Travel|      2024-11-11|          Cash|
|          1020|       C020|   218|  Fur

In [45]:
df_new=df_new.sort(col('transaction_id').asc())
df_new.show()


+--------------+-----------+------+-----------+----------------+--------------+
|transaction_id|customer_id|amount|   category|transaction_date|payment_method|
+--------------+-----------+------+-----------+----------------+--------------+
|          1001|       C001|   168|Electronics|      2024-06-22|    Debit Card|
|          1002|       C002|   826|  Groceries|      2024-07-23|        PayPal|
|          1003|       C003|   690|   Clothing|      2024-08-24|          Cash|
|          1004|       C004|   346|     Travel|      2024-09-25| Bank Transfer|
|          1005|       C005|   622|     Health|      2024-10-26|   Credit Card|
|          1006|       C006|  1006|  Furniture|      2024-11-27|    Debit Card|
|          1007|       C007|   540|      Books|      2024-12-28|        PayPal|
|          1008|       C008|   730|Electronics|      2024-01-01|          Cash|
|          1009|       C009|   900|  Groceries|      2024-02-02| Bank Transfer|
|          1010|       C010|   176|   Cl

In [46]:
df_new.dropDuplicates().show()

+--------------+-----------+------+-----------+----------------+--------------+
|transaction_id|customer_id|amount|   category|transaction_date|payment_method|
+--------------+-----------+------+-----------+----------------+--------------+
|          1044|       C044|  1005|  Groceries|      2024-01-09| Bank Transfer|
|          1224|       C024|   568|      Books|      2024-01-21| Bank Transfer|
|          1234|       C034|   505|   Clothing|      2024-11-03| Bank Transfer|
|          1278|       C078|   253|     Health|      2024-07-19|          Cash|
|          1764|       C064|    67|Electronics|      2024-01-01| Bank Transfer|
|          1820|       C020|   126|Electronics|      2024-09-01|   Credit Card|
|          1432|       C032|   364|     Health|      2024-05-05|        PayPal|
|          1694|       C094|   308|Electronics|      2024-03-15| Bank Transfer|
|          1876|       C076|   694|Electronics|      2024-05-01|    Debit Card|
|          1090|       C090|   804|  Fur

In [47]:
df_new.printSchema()

root
 |-- transaction_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- amount: string (nullable = true)
 |-- category: string (nullable = true)
 |-- transaction_date: string (nullable = true)
 |-- payment_method: string (nullable = true)



In [48]:
# prompt: print number of rows in df_new

print(df_new.count())


1000


In [52]:
df_money=df_new.groupBy(col('category'),col('payment_method')).agg(sum('amount').alias('Total_amount'))\
               .sort([col('category'),col('payment_method')],ascending=[1,1])
df_money.show()

+-----------+--------------+------------+
|   category|payment_method|Total_amount|
+-----------+--------------+------------+
|      Books| Bank Transfer|     16304.0|
|      Books|          Cash|     13405.0|
|      Books|   Credit Card|     15587.0|
|      Books|    Debit Card|     13292.0|
|      Books|        PayPal|     12937.0|
|   Clothing| Bank Transfer|     14263.0|
|   Clothing|          Cash|     14602.0|
|   Clothing|   Credit Card|     14981.0|
|   Clothing|    Debit Card|     13768.0|
|   Clothing|        PayPal|     20080.0|
|Electronics| Bank Transfer|     12571.0|
|Electronics|          Cash|     14593.0|
|Electronics|   Credit Card|     13738.0|
|Electronics|    Debit Card|     17474.0|
|Electronics|        PayPal|     13361.0|
|  Furniture| Bank Transfer|     14130.0|
|  Furniture|          Cash|     16779.0|
|  Furniture|   Credit Card|     13847.0|
|  Furniture|    Debit Card|     14970.0|
|  Furniture|        PayPal|     13897.0|
+-----------+--------------+------

In [57]:
df_money.write.format("csv").option("header", "true").save("/df_money.csv")